In [15]:
# Import required libraries
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

In [16]:
df = pd.read_csv('Data/student-scores-enhanced.csv')

In [17]:
df['performance_category'] = pd.cut(
    df['average_score'],
    bins=[0, 60, 75, 90, 100],
    labels=['Low', 'Average', 'High', 'Excellent']
)

In [18]:
# Encode the categorical target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['performance_category'])

In [19]:
# Select only numeric predictors (exclude target)
X = df.select_dtypes(include=['int64', 'float64']).drop(columns=['average_score'])

In [20]:
# Apply ANOVA F-test (SelectKBest)
#     - f_classif = ANOVA F-value between label/feature
#     - k = number of top features you want (change to 10, 15, etc.)
selector = SelectKBest(score_func=f_classif, k=15)
X_new = selector.fit_transform(X, y)

In [21]:
# Extract and display the selected features with their F-scores
selected_features = X.columns[selector.get_support()]
feature_scores = selector.scores_[selector.get_support()]

anova_results = pd.DataFrame({
    'Feature': selected_features,
    'F_Score': feature_scores
}).sort_values(by='F_Score', ascending=False)

print("===== ANOVA Feature Selection Results =====")
print(anova_results)

===== ANOVA Feature Selection Results =====
                             Feature     F_Score
11                       science_avg  586.306170
12                    humanities_avg  373.942324
14           performance_consistency  251.109936
1            weekly_self_study_hours  206.844445
13                  engagement_score  202.455732
10          career_aspiration_Doctor  174.005745
9   career_aspiration_Business Owner  169.406493
2                         math_score  166.443564
3                      history_score  131.533033
7                      english_score  129.145251
6                      biology_score  125.772410
5                    chemistry_score  118.196736
4                      physics_score  112.193688
8                    geography_score   82.755491
0                       absence_days   41.902852


In [22]:
# Save the reduced dataset for modeling
selected_cols = selected_features.tolist() + ['average_score', 'performance_category']
df[selected_cols].to_csv("student_scores_selected_features.csv", index=False)
print("\m Saved reduced dataset as 'student_scores_selected_features.csv'")



✅ Saved reduced dataset as 'student_scores_selected_features.csv'


In [11]:
df = pd.read_csv("Data/student_scores_selected_features.csv")
print("Number of columns:", len(df.columns))
print("Columns:", df.columns.tolist())

Number of columns: 17
Columns: ['absence_days', 'weekly_self_study_hours', 'math_score', 'history_score', 'physics_score', 'chemistry_score', 'biology_score', 'english_score', 'geography_score', 'career_aspiration_Business Owner', 'career_aspiration_Doctor', 'science_avg', 'humanities_avg', 'engagement_score', 'performance_consistency', 'average_score', 'performance_category']


In [12]:
#Chi-Square Feature Selection (Significant Features Only for ML)

import pandas as pd
from scipy.stats import chi2_contingency

#Load dataset
df = pd.read_csv("Data/student-scores-enhanced.csv")

#Create performance category (target variable)
df["performance_group"] = pd.qcut(df["average_score"], q=3, labels=["Low", "Medium", "High"])

#Define categorical and numeric features for Chi-Square
categorical_features = ["gender", "part_time_job", "extracurricular_activities"]
numeric_to_discretize = ["absence_days", "weekly_self_study_hours", "engagement_score"]

#Discretize numeric features into quantile-based bins (Low, Medium, High)
for col in numeric_to_discretize:
    df[f"{col}_binned"] = pd.qcut(df[col], q=3, labels=["Low", "Medium", "High"])

#Combine all categorical-like features for testing
all_features = categorical_features + [f"{col}_binned" for col in numeric_to_discretize]

#Run Chi-Square tests
chi_results = []
for col in all_features:
    contingency = pd.crosstab(df[col], df["performance_group"])
    chi2, p, dof, expected = chi2_contingency(contingency)
    chi_results.append({
        "Feature": col,
        "Chi-Square": chi2,
        "p-value": p,
        "Significance": (
            "Highly Significant" if p < 0.001
            else "Significant" if p < 0.05
            else "Not Significant"
        )
    })

chi_df = pd.DataFrame(chi_results).sort_values("p-value")
print("=== 📊 Chi-Square Test Results ===")
display(chi_df)

#Select only significant features (p < 0.05)
significant_features = chi_df[chi_df["p-value"] < 0.05]["Feature"].tolist()

print("\n✅ Significant Features (p < 0.05):")
for f in significant_features:
    print(f"-", f)

#Replace '_binned' with original numeric columns for ML dataset
final_features = []
for f in significant_features:
    if f.endswith("_binned"):
        final_features.append(f.replace("_binned", ""))
    else:
        final_features.append(f)

#Create final reduced dataset (only selected features + target)
reduced_df = df[final_features + ["performance_group"]]

#Save the reduced dataset
output_path = "student_scores_chi_selected.csv"
reduced_df.to_csv(output_path, index=False)

print(f"\n💾 Reduced dataset saved as '{output_path}'")
print(f"➡️ Shape: {reduced_df.shape}")
print(f"➡️ Final Features: {final_features}")


=== 📊 Chi-Square Test Results ===


,Feature,Chi-Square,p-value,Significance
4,weekly_self_study_hours_binned,274.919604,2.775174e-58,Highly Significant
5,engagement_score_binned,256.186751,3.024530e-54,Highly Significant
3,absence_days_binned,54.573338,3.991463e-11,Highly Significant
1,part_time_job,41.450329,9.981031e-10,Highly Significant
2,extracurricular_activities,2.069021,3.554003e-01,Not Significant
0,gender,0.124819,9.394982e-01,Not Significant



✅ Significant Features (p < 0.05):
- weekly_self_study_hours_binned
- engagement_score_binned
- absence_days_binned
- part_time_job

💾 Reduced dataset saved as 'student_scores_chi_selected.csv'
➡️ Shape: (2000, 5)
➡️ Final Features: ['weekly_self_study_hours', 'engagement_score', 'absence_days', 'part_time_job']
